In [1]:
import os
import pandas as pd
from Bio import SeqIO
import multiprocessing as mp

In [2]:
##load chimeras from a previous iteration of the pipeline
previous_iteration = SeqIO.to_dict(SeqIO.parse('outputs/previous_iteration_chimeras.fa', 'fasta'))

In [3]:
#load fasta w/ mmseqs clustering results 
record_dictc = SeqIO.to_dict(SeqIO.parse('outputs/mmseq_cluster_representatives.fasta', 'fasta'))

In [4]:
## define the set of chimeras missing from mmseqs clustering this iteration
missing_previous_iteration=set(previous_iteration)-set(record_dictc)

In [6]:
missing_previous_iteration

{'GCF_001856785.1;XP_022173178.1',
 'GCF_002204515.2;XP_021699539.1',
 'GCF_004354385.1;XP_034487048.1',
 'GCF_015228065.2;XP_037790819.1',
 'GCF_021134715.1;XP_046439036.1',
 'GCF_021134715.1;XP_046456339.1',
 'GCF_021234035.1;XP_046649021.1',
 'GCF_024362695.1;XP_049881676.1',
 'GCF_902806685.2;XP_069356709.1',
 'GCF_905147365.1;XP_041972388.1',
 'GCF_907165205.1;XP_044763649.1'}

In [9]:
combined_records = list(record_dictc.values())                    # keep the current representatives first
combined_records.extend(previous_iteration[_id] for _id in missing_previous_iteration)  # then append the extras

out_path = "outputs/mmseq_cluster_representatives_with_missing.fasta"
SeqIO.write(combined_records, out_path, "fasta")

610359

In [13]:
##load fasta with all input sequences 
record_dict = SeqIO.to_dict(SeqIO.parse('outputs/concatenated_filtered_proteins.fa', 'fasta'))

In [4]:
## set of sequences that successfully ran with diamond blast round 1
blast_outputs=set(os.listdir("/n/netscratch/extavour_lab/Everyone/Rishabh/round1_diamond_split_outputs/"))
blast_outputs=set([x.replace(".tsv","") for x in blast_outputs])

In [19]:
missing_blast=set(record_dictc.keys())-blast_outputs

In [36]:
def write_fasta_entry(key):
    with open("outputs/missing_blast_hits_round1.fasta", "a") as output_handle:
        SeqIO.write(record_dict[key], output_handle, "fasta")

In [30]:
len(missing_blast), len(missing_previous_iteration)

(33481, 11)

In [31]:
tw=missing_blast|missing_previous_iteration

In [37]:
with mp.Pool(processes=20) as pool:
    
    result=pool.map(write_fasta_entry, tw)

In [39]:
odict = SeqIO.to_dict(SeqIO.parse('outputs/missing_blast_hits_round1.fasta', 'fasta'))

In [42]:
!sbatch scripts/run_diamond_round1_on_missing_fasta.sh

Submitted batch job 12944297


In [1]:
!sbatch scripts/split_blast_table.sh /n/netscratch/extavour_lab/Everyone/Rishabh/round1_diamond_out/missing.tsv "/n/netscratch/extavour_lab/Everyone/Rishabh/round1_diamond_split_outputs"

Submitted batch job 13029705
